In [1]:
!pip install -q transformers torch
!pip install datasets

!pip uninstall transformers
!pip install --no-cache-dir transformers sentencepiece
!pip install rouge_score
!pip install evaluate

     |████████████████████████████████| 5.5 MB 26.6 MB/s 
     |████████████████████████████████| 182 kB 57.4 MB/s 
     |████████████████████████████████| 7.6 MB 90.0 MB/s 
ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 27.2 MB/s 
     |████████████████████████████████| 212 kB 92.8 MB/s 
     |████████████████████████████████| 115 kB 57.1 MB/s 
     |████████████████████████████████| 127 kB 92.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Found existing installation: transformers 4.24.0
Uninstalling transformers-4.24.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/transformers-4.24.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/transformers/*
Proceed (y/n)? y
  Successfully uninstalled transformers-4.24.0
Looki

In [6]:
from transformers import TFAutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from datasets import load_dataset
from datasets import concatenate_datasets, DatasetDict

from datasets import load_dataset
import pandas as pd
import torch
from transformers.tokenization_utils_base import BatchEncoding
from typing import List, Tuple
from tqdm import tqdm
from transformers import AdamW, get_linear_schedule_with_warmup
from datasets import load_metric
import tensorflow as tf
import os
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
from nltk.tokenize import sent_tokenize
import nltk
import evaluate
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
df = load_dataset('csv', data_files = {'train': 'sample_data/cnn_train_class1.csv',
                                                              'validation':'sample_data/cnn_val_class1.csv',
                                                              'test':'sample_data/cnn_test_class1.csv'})
df

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b17cd92abcae5e27/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'article', 'highlights', 'class'],
        num_rows: 71327
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'article', 'highlights', 'class'],
        num_rows: 5041
    })
    test: Dataset({
        features: ['Unnamed: 0', 'article', 'highlights', 'class'],
        num_rows: 4337
    })
})

In [8]:
ds_train = df['train']

In [9]:
ds_train['class']

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [10]:
ds_val = df['validation']

In [11]:
ds_test = df['test']

In [12]:
ds_train

Dataset({
    features: ['Unnamed: 0', 'article', 'highlights', 'class'],
    num_rows: 71327
})

In [13]:
ds_val

Dataset({
    features: ['Unnamed: 0', 'article', 'highlights', 'class'],
    num_rows: 5041
})

In [14]:
ds_test

Dataset({
    features: ['Unnamed: 0', 'article', 'highlights', 'class'],
    num_rows: 4337
})

In [15]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [16]:
model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/553 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


Downloading:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [22]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [23]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
    
Token: 
Add token as git credential? (Y/n) y
Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [24]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [25]:
max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["article"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["highlights"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [26]:
tokenized_train = ds_train.map(preprocess_function, batched=True)
tokenized_val =ds_val.map(preprocess_function, batched=True)
tokenized_test =ds_test.map(preprocess_function, batched=True)


  0%|          | 0/72 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [27]:
tokenized_train=tokenized_train.remove_columns(ds_train.column_names)
tokenized_test=tokenized_test.remove_columns(ds_test.column_names)
tokenized_val=tokenized_val.remove_columns(ds_val.column_names)

features = [tokenized_train[i] for i in range(2)]

In [28]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_train,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=8)
  
tf_eval_dataset = model.prepare_tf_dataset(
    tokenized_val,
    collate_fn=data_collator,
    shuffle=False,
    batch_size=8)
  

num_train_epochs = 8
num_train_steps = len(tf_train_dataset) * num_train_epochs
model_name = model_checkpoint.split("/")[-1]

optimizer, schedule = create_optimizer(
    init_lr=5.6e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01)


model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [30]:
callback = PushToHubCallback(
    output_dir=f"{model_name}-finetuned-mt5-class1", tokenizer=tokenizer
)

model.fit(
    tf_train_dataset, validation_data=tf_eval_dataset, callbacks=[callback], epochs=8
)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/repository.py:729: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.12. Please create the repository first using `create_repo(..., exists_ok=True)`.
  FutureWarning,
Cloning https://huggingface.co/ririying/mt5-small-finetuned-mt5-class1 into local empty directory.


Epoch 1/8
8915/8915 [==============================] - ETA: 0s - loss: 3.8999

Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


8915/8915 [==============================] - 977s 107ms/step - loss: 3.8999 - val_loss: 2.2395
Epoch 2/8
8915/8915 [==============================] - 939s 105ms/step - loss: 2.6457 - val_loss: 1.9951
Epoch 3/8
8915/8915 [==============================] - 935s 105ms/step - loss: 2.3865 - val_loss: 1.8784
Epoch 4/8
8915/8915 [==============================] - 924s 104ms/step - loss: 2.2622 - val_loss: 1.8179
Epoch 5/8
8915/8915 [==============================] - 927s 104ms/step - loss: 2.1877 - val_loss: 1.7959
Epoch 6/8
8915/8915 [==============================] - 926s 104ms/step - loss: 2.1395 - val_loss: 1.7820
Epoch 7/8
8915/8915 [==============================] - 927s 104ms/step - loss: 2.1085 - val_loss: 1.7720
Epoch 8/8
8915/8915 [==============================] - 928s 104ms/step - loss: 2.0908 - val_loss: 1.7689


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file tf_model.h5:   0%|          | 3.33k/2.07G [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/ririying/mt5-small-finetuned-mt5-class1
   6d7b41a..12e92cc  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/ririying/mt5-small-finetuned-mt5-class1
   6d7b41a..12e92cc  main -> main



In [31]:
from tqdm import tqdm
import numpy as np

generation_data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=320
)

tf_generate_dataset = model.prepare_tf_dataset(
    tokenized_val,
    collate_fn=generation_data_collator,
    shuffle=False,
    batch_size=8,
    drop_remainder=True,
)

In [32]:
@tf.function(jit_compile=True)
def generate_with_xla(batch):
    return model.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=32,
    )

In [33]:
all_preds = []
all_labels = []
for batch, labels in tqdm(tf_generate_dataset):
    predictions = generate_with_xla(batch)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = labels.numpy()
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    all_preds.extend(decoded_preds)
    all_labels.extend(decoded_labels)

100%|██████████| 630/630 [01:15<00:00,  8.36it/s]


In [34]:
rouge_score = evaluate.load("rouge")
result = rouge_score.compute(
    predictions=decoded_preds, references=decoded_labels, use_stemmer=True
)
result = {key: value* 100 for key, value in result.items()}
{k: round(v, 4) for k, v in result.items()}

{'rouge1': 40.5733, 'rouge2': 19.831, 'rougeL': 37.7023, 'rougeLsum': 37.1794}

#### Rouge score for test

In [35]:
from tqdm import tqdm
import numpy as np

generation_data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=320
)

tf_generate_dataset = model.prepare_tf_dataset(
    tokenized_test,
    collate_fn=generation_data_collator,
    shuffle=False,
    batch_size=8,
    drop_remainder=True,
)

In [36]:
@tf.function(jit_compile=True)
def generate_with_xla(batch):
    return model.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=32,
    )

In [37]:
all_preds = []
all_labels = []
for batch, labels in tqdm(tf_generate_dataset):
    predictions = generate_with_xla(batch)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = labels.numpy()
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    all_preds.extend(decoded_preds)
    all_labels.extend(decoded_labels)

100%|██████████| 542/542 [01:00<00:00,  8.92it/s]


In [38]:
rouge_score = evaluate.load("rouge")
result = rouge_score.compute(
    predictions=decoded_preds, references=decoded_labels, use_stemmer=True
)
result = {key: value* 100 for key, value in result.items()}
{k: round(v, 4) for k, v in result.items()}

{'rouge1': 42.2708, 'rouge2': 22.0767, 'rougeL': 37.7243, 'rougeLsum': 38.8421}

In [39]:
import pandas as pd
df_class1_test = pd.DataFrame(all_preds)

In [40]:
df_test = pd.DataFrame(df['test'])

In [41]:
df_test.head(), len(df_test)

(   Unnamed: 0                                            article  \
 0           1  (CNN)Never mind cats having nine lives. A stra...   
 1           4  (CNN)A Duke student has admitted to hanging a ...   
 2           5  (CNN)He's a blue chip college basketball recru...   
 3           7  (CNN)Andrew Getty, one of the heirs to billion...   
 4          20  Norfolk, Virginia (CNN)The second mate of the ...   
 
                                           highlights  class  
 0  Theia, a bully breed mix, was apparently hit b...      1  
 1  Student is no longer on Duke University campus...      1  
 2  College-bound basketball star asks girl with D...      1  
 3  Andrew Getty's death appears to be from natura...      1  
 4  Father: "I know he went through what he went t...      1  , 4337)

In [42]:
df_test_new = df_test.iloc[:len(df_test), 1:]
df_test_new.head(), len(df_test_new)

(                                             article  \
 0  (CNN)Never mind cats having nine lives. A stra...   
 1  (CNN)A Duke student has admitted to hanging a ...   
 2  (CNN)He's a blue chip college basketball recru...   
 3  (CNN)Andrew Getty, one of the heirs to billion...   
 4  Norfolk, Virginia (CNN)The second mate of the ...   
 
                                           highlights  class  
 0  Theia, a bully breed mix, was apparently hit b...      1  
 1  Student is no longer on Duke University campus...      1  
 2  College-bound basketball star asks girl with D...      1  
 3  Andrew Getty's death appears to be from natura...      1  
 4  Father: "I know he went through what he went t...      1  , 4337)

In [43]:
df_test_new['summary'] = df_class1_test
df_test_new.head()

,article,highlights,class,summary
0,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...",1,Theia was taken in by a worker who took her to...
1,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,1,Duke student admitted to hanging noose on the ...
2,(CNN)He's a blue chip college basketball recru...,College-bound basketball star asks girl with D...,1,Trey Moses and Ellie Meredith couldn't be more...
3,"(CNN)Andrew Getty, one of the heirs to billion...",Andrew Getty's death appears to be from natura...,1,"Andrew Getty, 47, has a net worth of $2.1 bill..."
4,"Norfolk, Virginia (CNN)The second mate of the ...","Father: ""I know he went through what he went t...",1,"Louis Jordan, 37, was moving on the 35-foot Pe..."


In [44]:
df_test_new.to_csv('test_class1_final.csv', index=False)

In [45]:
from google.colab import files
files.download('/content/test_class1_final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
!cp test_class1_final.csv "drive/My Drive/580project"

In [ ]:
from transformers import pipeline

hub_model_id = "ririying/mt5-small-finetuned-test"
summarizer = pipeline("summarization", model=hub_model_id)

In [ ]:
!huggingface-cli login

In [ ]:
model.push_to_hub("ririying/my-finetuned-mt5-class1")

In [ ]:
# use the fine-tune model
#from transformers import pipeline

#hub_model_id = 'SiriRRR/mt5-small-finetuned-test'
#summarizer = pipeline("summarization", model=hub_model_id)